# Data Cleaning

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Cleaning:-Lyrics-DataFrame" data-toc-modified-id="Cleaning:-Lyrics-DataFrame-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cleaning: Lyrics DataFrame</a></span></li><li><span><a href="#Database-&amp;-Data-Dictionary" data-toc-modified-id="Database-&amp;-Data-Dictionary-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Database &amp; Data Dictionary</a></span><ul class="toc-item"><li><span><a href="#Postgres-Database:-capstone" data-toc-modified-id="Postgres-Database:-capstone-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Postgres Database: capstone</a></span></li><li><span><a href="#Data-Dictionary-of-Capstone-Database" data-toc-modified-id="Data-Dictionary-of-Capstone-Database-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Data Dictionary of Capstone Database</a></span></li></ul></li></ul></div>

### Imports

In [1]:
import pandas as pd
import numpy as np
import re
import unidecode
import seaborn as sns
import matplotlib as plt
import string

### Functions

In [2]:
def wrong_artist_index(dataframe, artists_list):
    '''
    This is a function which will receive a dataframe, here the freshly scrapped dataframe,
    and the list of curated artists as arguments and return the indexes of artists in the dataframe that
    are not found in the artists list.
    '''
    unique_artists = list(dataframe.artist_name.unique())
    
    wrong_artists = []
    for i,j in zip([x.lower() for x in unique_artists],unique_artists):
            if i not in [x.lower() for x in artists_list]:
                wrong_artists.append(j)
    return list(dataframe[dataframe.artist_name.isin(wrong_artists)].index)

In [3]:
def check_spelling(dataframe, artists_list):
    unique_artists = list(dataframe.artist_name.unique())
    
    false_spelling = []
    for i in unique_artists:
            if i not in artists_list:
                false_spelling.append(i)
    return false_spelling

### Cleaning: Lyrics DataFrame

* Removing duplicated entries

* Adjust artist name spellings of the Genius Data to the Spotify spelling

* Removing entries with more than primary artists or unwanted artists

* Dropping genius_url column

In [4]:
lyrics_df = pd.read_csv('/Users/constancemaurer/GA DSI 12/DSI12-lessons/projects/project-capstone/personal-github/Resources/lyrics_df.csv')
spotify = pd.read_csv('/Users/constancemaurer/GA DSI 12/DSI12-lessons/projects/project-capstone/personal-github/Resources/Spotify_Kaggle_noduplicates.csv')

In [5]:
artists = ['John Legend','Adele','Frank Ocean','Aretha Franklin','Amy Winehouse','The Rolling Stones',
           'Led Zeppelin','Arctic Monkeys','The Beatles','Queen','Fall Out Boy','Tame Impala','Beach House',
          'Kings of Leon','Radiohead','Red Hot Chilli Peppers','Gorillaz','Linkin Park','The Black Keys','Slipknot',
          'Metallica','Korn','Nirvana','Foo Fighters','The Neighborhood','The Weeknd','Kayne West','Beyoncé',
          'Rihanna','Shine Down','Kendrick Lamar','Florence & The Machine','Tyler, The Creator','A$AP Rocky',
          'Lil Wayne','System Of A Down','Passion Pit','Caribou','Twin Shadow','HAIM','Sister Sledge',
           'Taylor Swift', 'Mac Miller', 'Red Hot Chili Peppers', 'The Weeknd', 'Fleetwood Mac', 'Wiz Khalifa', 
           'Lady Gaga', 'LCD Soundsystem']

**Removing duplicates with .drop_duplicates( )**

In [6]:
lyrics_df.drop_duplicates(inplace=True)

**Removing entries with more than primary artists or unwanted artists**

In [7]:
wrong_art_index = wrong_artist_index(lyrics_df, artists)
wrong_art_index[:5]

[35, 356, 374, 377, 378]

In [8]:
lyrics_df.drop(index=wrong_art_index, inplace=True)

In [9]:
lyrics_df.head()

,track_name,artist_name,release_year,spotify_uri,genius_url,lyrics
0,Distorted Records,A$AP Rocky,2018-05-25,NaN,NaN,"[Intro]\nUh, uh\n\n[Pre-Chorus 1]\nI can feel ..."
1,Jukebox Joints,A$AP Rocky,2015-05-26,spotify:track:2gAGWaK4wvt2xrFUlR4mK8,NaN,[Part I]\n\n[Chorus: A$AP Rocky & Joe Fox]\nAn...
2,West Side Highway,A$AP Rocky,2015-05-26,spotify:track:3RySVukMPe4R7lAP8oGI17,NaN,[Chorus: A$AP Rocky]\nThese days of preference...
3,1Train,A$AP Rocky,2013-01-15,spotify:track:223cmlU9Vd2vNTlfMdWCy4,NaN,[Produced by HitBoy]\n\n[Verse 1: A$AP Rocky]\...
4,Fashion Killa,A$AP Rocky,2013-01-15,spotify:track:40H5libEZWrbkc8HTlXGbt,NaN,"[Chorus]\nHer pistol go…\n(Doot-doot-doot, ban..."


In [10]:
lyrics_df.shape

(1688, 6)

**Adjust artist name spellings of the Genius Data to the Spotify spelling**

* Checked which artists are spelled incorrectly by using the check_spelling( ) function (see Functions sections).

In [11]:
check_spelling(lyrics_df, artists)

['System of a Down']

* Changed Genius' 'System of a Down' to 'System Of A Down'.

In [12]:
lyrics_df.artist_name = lyrics_df.artist_name.apply(lambda x : x.replace('System of a Down','System Of A Down'))

In [13]:
lyrics_df[lyrics_df.artist_name=='System of a Down'].head(3)

,track_name,artist_name,release_year,spotify_uri,genius_url,lyrics


In [14]:
lyrics_df[lyrics_df.artist_name=='System Of A Down'].head(3)

,track_name,artist_name,release_year,spotify_uri,genius_url,lyrics
1576,Sugar,System Of A Down,1998-05-24,spotify:track:31RTFPrB7wmYBhlkM2ILXG,NaN,[Chorus: Serj Tankian & Daron Malakian]\nThe K...
1577,Lonely Day,System Of A Down,2005-11-22,spotify:track:1VNWaY3uNfoeWqb5U8x2QX,NaN,[Verse 1: Daron Malakian]\nSuch a lonely day a...
1578,Radio/Video,System Of A Down,2005-05-17,NaN,NaN,[Intro: Serj Tankian]\nLa la la la la la\n\n[C...


**Dropping the genius_url column**

Dropped the genius_url column as the initial code I wrote did not work for the majority of artist (JSON files weren't structured reliably for me). Please try it yourself :)

In [15]:
lyrics_df.drop(columns=['genius_url'], inplace=True)

In [16]:
lyrics_df.head()

,track_name,artist_name,release_year,spotify_uri,lyrics
0,Distorted Records,A$AP Rocky,2018-05-25,NaN,"[Intro]\nUh, uh\n\n[Pre-Chorus 1]\nI can feel ..."
1,Jukebox Joints,A$AP Rocky,2015-05-26,spotify:track:2gAGWaK4wvt2xrFUlR4mK8,[Part I]\n\n[Chorus: A$AP Rocky & Joe Fox]\nAn...
2,West Side Highway,A$AP Rocky,2015-05-26,spotify:track:3RySVukMPe4R7lAP8oGI17,[Chorus: A$AP Rocky]\nThese days of preference...
3,1Train,A$AP Rocky,2013-01-15,spotify:track:223cmlU9Vd2vNTlfMdWCy4,[Produced by HitBoy]\n\n[Verse 1: A$AP Rocky]\...
4,Fashion Killa,A$AP Rocky,2013-01-15,spotify:track:40H5libEZWrbkc8HTlXGbt,"[Chorus]\nHer pistol go…\n(Doot-doot-doot, ban..."


In [17]:
lyrics_df.dtypes

track_name      object
artist_name     object
release_year    object
spotify_uri     object
lyrics          object
dtype: object

In [18]:
lyrics_df.shape

(1688, 5)

In [20]:
lyrics_df.to_csv(r'lyrics_table', index=False)

### Database & Data Dictionary

* Generating an SQL database for General Assembly Capstone project

* Adding spotify and genius datasets to the database

* Creating SQL and pandas query functions for easier navigation

* Merging the lyrics and spotify dataset to form model's dataframe

* Creating a data dictionary for the raw data and adding it to the SQL database

#### Postgres Database: capstone

```python
import sqlalchemy
import psycopg2
import pandas as pd
from pandas.io import sql

# DSN (data source name) format for database connections:  
# [protocol / database  name]://[username]:[password]@[hostname / ip]:[port]/[database name here]


# on your computer you are the user postgres (full administrative access)
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to creat a database with the following name
database = 'gacapstone'
#set up a connection to postgres without specifying the database
conn_postgres = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}'```

In [21]:
import sqlalchemy
import psycopg2
import pandas as pd
from pandas.io import sql

# DSN (data source name) format for database connections:  
# [protocol / database  name]://[username]:[password]@[hostname / ip]:[port]/[database name here]


# on your computer you are the user postgres (full administrative access)
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to creat a database with the following name
database = 'gacapstone'
#set up a connection to postgres without specifying the database
conn_postgres = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}'

*Create an sqlalchemy connection:*

```python
engine = sqlalchemy.engine.create_engine(conn_postgres)```

*Connect using sqlalchemy:*

```python
connection = engine.connect()```

*Commit connection:*

```python
connection.execute('commit')```

*Create database:*

```python
connection.execute(f"create database {database}")```

*Close connection:*

```python
connection.close()```

* Opened a new connection to the database & add tables.

In [22]:
conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)

*Connect using sqlalchemy:*

In [23]:
engine = sqlalchemy.engine.create_engine(conn_str)

In [24]:
spotify_df = pd.read_csv('/Users/constancemaurer/GA DSI 12/DSI12-lessons/projects/project-capstone/personal-github/Resources/Spotify_Kaggle_noduplicates.csv')

spotify_df.to_sql(name = 'spotify_tracks', con = engine, if_exists = 'replace', index = False)

In [25]:
conn.commit()

In [26]:
query = '''SELECT * FROM spotify_tracks'''

df = pd.read_sql(query, con=conn)
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [27]:
genius_table= pd.read_csv('/Users/constancemaurer/GA DSI 12/DSI12-lessons/projects/project-capstone/personal-github/Resources/lyrics_table')

genius_table.to_sql(name = 'genius_lyrics', con = engine, if_exists = 'replace', index = False)

In [28]:
conn.commit()

**Function to query sql more flexibily and return SQL or pandas dataframe:**

In [29]:
def sql_Q(query, conn=conn):
    '''This function queries SQL databases by 
    accepting a query string and the connection.
    It returns an SQL table.'''
    return sql.read_sql(query, con=conn)

def pd_Q(query, conn=conn):
    '''This function queries SQL databases for tables by 
    accepting a query string and the connection.
    It returns a pandas dataframe.'''
    return pd.read_sql(query, con=conn)

* Extracting tables as pandas dataframes.

In [30]:
g = pd_Q('''SELECT * FROM genius_lyrics 
        ORDER BY artist_name''')

In [31]:
s = pd_Q('''SELECT * FROM spotify_tracks
             ORDER BY artist_name''')

* Merging the genius_lyrics (g) with the matching spotify_tracks (s) rows.

In [32]:
new = g.merge(s, how='left', left_on=['track_name', 'artist_name'], right_on=['track_name', 'artist_name'])

In [33]:
new.to_csv('artist_predictor_RAW.csv', index=False)

#### Data Dictionary of Capstone Database

For the data dictionary, I created a dataframe and then added it as a table to the postgres capstone database.
The features/variables of the original spotify dataset, sourced from a Kaggle competition (https://www.kaggle.com/zaheenhamidani/ultimate-spotify-tracks-db/data), I described using spotify's own documentation given here (https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/). The data dictionary contains both the spotify_tracks table and the genius_lyrics table's variables. The data dictionary contains information on:
    - variable name
    - description of the variable
    - data type of the variable
    - location (referring to which table) the variable belongs to

* Extracted the variable names by appending the column names of the two tables to a list. From genius_lyrics table I excluded the track_name and artist_name as these are contained in both tables.

In [34]:
variables = []

In [35]:
for item in pd_Q('''SELECT * FROM spotify_tracks
        LIMIT 1''').columns:
    variables.append(item)

In [36]:
for item in pd_Q('''SELECT * FROM genius_lyrics
        LIMIT 1''').columns[2:]:
    variables.append(item)

In [37]:
variables

['genre',
 'artist_name',
 'track_name',
 'track_id',
 'popularity',
 'acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'speechiness',
 'tempo',
 'time_signature',
 'valence',
 'release_year',
 'spotify_uri',
 'lyrics']

* Defined variables matching the column names to write their description as a string. I created a dictionary with the variable names as one entry and appende the descrptions as another entry.

In [38]:
genre = 'The music genre of the song.'
artist_n = 'The artist or band name of the song.'
track_n = 'The track name is simply title of the song.'
track_id = 'The spotify ID number which at the time the dataset was collected functioned as the uri but might not match with current uris and IDs.'
popularity = 'Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time.'
acousticness = 'A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.'
danceability = 'Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.'
duration_ms = 'The duration of the track in milliseconds.'
energy = 'Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.'
instrumentalness = 'Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.'
key = 'The estimated overall key of the track. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.'
liveness = 'Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.'
loudness = 'The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.'
mode = 'Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.'
speechiness = 'Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.'
tempo = 'The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.'
time_signature = 'An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure).'
valence = 'A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).'
release_year = 'The date the track/song was released.'
spotify_uri = "Genius' record of spotify uri for a track."
lyrics = 'Web scrapped lyrics from genius.com website.'

In [39]:
data_dictionary = {'Variable': variables}

In [41]:
data_dictionary['Description']= [genre, artist_n, track_n, track_id, popularity, acousticness, danceability, 
                                 duration_ms, energy, instrumentalness, key, liveness, loudness, mode,
                                 speechiness, tempo, time_signature, valence, release_year, spotify_uri, lyrics]
                                  

* Checked all the current data types of the variables and added them to the data dictionary.

In [42]:
sql_Q('''SELECT * FROM spotify_tracks 
        ORDER BY artist_name
        LIMIT 1''').dtypes

genre                object
artist_name          object
track_name           object
track_id             object
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo               float64
time_signature       object
valence             float64
dtype: object

In [43]:
sql_Q('''SELECT * FROM genius_lyrics 
        LIMIT 1''').dtypes

track_name      object
artist_name     object
release_year    object
spotify_uri     object
lyrics          object
dtype: object

In [44]:
data_dictionary['Data Type'] = ['string object','string object','string object', 'string object', 'int64',
                                'float64','float64','int64','float64','float64','string object','float64',
                               'float64','string object','float64','float64','string object','float 64',
                               'string object - has to be changed to datetime','string object','string object']

* Added the location of each variables to the data dictionary.

In [45]:
locations = [['spotify_track', 'genius_lyrics'], ['spotify_track', 'genius_lyrics']]

for i in range(0+2,len(pd_Q('''SELECT * FROM spotify_tracks LIMIT 1''').columns)):
    locations.append(['spotify_track'])
    
for i in range (0+2,len(pd_Q('''SELECT * FROM genius_lyrics LIMIT 1''').columns)):
    locations.append(['genius_lyrics'])

In [46]:
data_dictionary['Location'] = locations

In [47]:
data_dict_df = pd.DataFrame(data_dictionary)
data_dict_df

,Variable,Description,Data Type,Location
0,genre,The music genre of the song.,string object,"[spotify_track, genius_lyrics]"
1,artist_name,The artist or band name of the song.,string object,"[spotify_track, genius_lyrics]"
2,track_name,The track name is simply title of the song.,string object,[spotify_track]
3,track_id,The spotify ID number which at the time the da...,string object,[spotify_track]
4,popularity,"Generally speaking, songs that are being playe...",int64,[spotify_track]
5,acousticness,A confidence measure from 0.0 to 1.0 of whethe...,float64,[spotify_track]
6,danceability,Danceability describes how suitable a track is...,float64,[spotify_track]
7,duration_ms,The duration of the track in milliseconds.,int64,[spotify_track]
8,energy,Energy is a measure from 0.0 to 1.0 and repres...,float64,[spotify_track]
9,instrumentalness,Predicts whether a track contains no vocals. “...,float64,[spotify_track]


* Added the data dictionary to the gacapstone database.

In [48]:
data_dict_df.to_sql(name = 'data_dictionary', con = engine, if_exists = 'replace', index = False)

In [49]:
conn.commit()

In [50]:
sql_Q('''SELECT * FROM data_dictionary ''')

,Variable,Description,Data Type,Location
0,genre,The music genre of the song.,string object,"{spotify_track,genius_lyrics}"
1,artist_name,The artist or band name of the song.,string object,"{spotify_track,genius_lyrics}"
2,track_name,The track name is simply title of the song.,string object,{spotify_track}
3,track_id,The spotify ID number which at the time the da...,string object,{spotify_track}
4,popularity,"Generally speaking, songs that are being playe...",int64,{spotify_track}
5,acousticness,A confidence measure from 0.0 to 1.0 of whethe...,float64,{spotify_track}
6,danceability,Danceability describes how suitable a track is...,float64,{spotify_track}
7,duration_ms,The duration of the track in milliseconds.,int64,{spotify_track}
8,energy,Energy is a measure from 0.0 to 1.0 and repres...,float64,{spotify_track}
9,instrumentalness,Predicts whether a track contains no vocals. “...,float64,{spotify_track}
